### Introduction


**Probelm Statement:** Make a classifier which takes in a job description and gives the department name for it.
*   Use a neural network model
*   Make use of a pre-trained Word Embeddings (example: Word2Vec, GloVe, etc.)
*   Calculate the accuracy on a test set (data not used to train the model)

**Problem Solving Approach:** 
_Provide a brief description of steps you followed for solving this problem_
1. Understanding the data and different classes to classify jobs into
2. Will try to take 75% jobs of each department to train and rest 25% to test
3. Output class will be a one hot vector 
4. Input will be observed attributes of a job description. Will use word2vec or universal sentence encoder to compute embedding of a sentence to feed as an attribute of a job description.
5. Starting with a simple 2 layer neural network to see how it performs.

### Part I: Text Preprocessing

_Include all text preprocesing steps like processing of json,csv files & data cleaning in this part._

Import neccessary packages in below cell

In [76]:
import json
with open('data/document_departments.csv') as f:
    content = f.readlines()


description = {}
for i in xrange(1,len(content)):
    k = content[i].split(',')[1][:-2].strip()
    if k in description.keys():
        description[k].append(content[i].split(',')[0])
    else:
        description[k] = []
        description[k].append(content[i].split(',')[0])
        
with open('data/docs/5922081.json', 'r') as myfile:
    data=str(myfile.read())

# parse file
obj = json.loads(data)
print(obj.keys())
print(obj['company_info']['Company Description'])
for k in obj.keys():
    print(k)
    if type(obj[k]) == 'dict':
        print(obj[k].keys())
    

[u'company_info', u'api_data', u'other_details', u'topbox_information', u'jd_information', u'_id']
cmts is engaged in the data entry outsourcing and supply of skilled/semi skilled &amp; unskilled Manpower across the industry spectrum. Our pool of manpower ranges from professionals such as Sales &amp; technical professionals in different categories, IT professionals, aviation sector, technical profiles, engineers, non-technical in-house professionals, administrative profiles, Accountant, HR, Back office, Data Entry Operator etc. Our organization's goal is total clients satisfaction through quality and personalized delivery of each and every service. We therefore exist to cater the need of these prospecting employers/companies since our inception in 2008.
company_info
api_data
other_details
topbox_information
jd_information
_id


In [97]:
import gensim

# Load Google's pre-trained Word2Vec model.
word_model = gensim.models.KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)  

In [71]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/anshul/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [182]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np

#Resusable functions keeping the problem and preprocessing of text in mind

def preprocess(sen):
    if sen == "":
        return ""
    
    if type(sen) is list:
        s = ""
        for word in sen:
            s += word + ' '
        sen = s[:-1]
    sen = sen.replace("&nbsp;", ' ')
    sen = sen.replace("&amp;", ' ')
    sen = sen.replace('.', ' ')
    sen = sen.replace(',', ' ')
    sen = sen.lower()
    sen = re.sub(r'\d+', '', sen)
    
    #sen = sen.translate(string.maketrans('','', string.punctuation))
    exclude = set(string.punctuation)
    for c in exclude:
        sen = sen.replace(c, ' ')
    sen = ''.join(ch for ch in sen if ch not in exclude)
    sen = sen.strip()
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(sen)
    sen = [i for i in tokens if not i in stop_words]
    return sen

def embed(sen):
    
    sen = preprocess(sen)
    #print(sen)
    emm = np.ones(300)/100
    em = np.zeros(300)
    count = 0
    for w in sen:
        try:
            em += word_model[w]
            count+=1
        except KeyError:
            print(w, "Not found")
    
    #print(em/count)
    if sen == "" or count == 0:
        return emm
    return em/count

def get_training_data(obj):
    #print obj['jd_information']['description']
    data = []
    try:
        data.append(embed(obj['company_info']['Company Description']))
    except KeyError:
        data.append(embed(""))
    try:
        data.append(embed(obj['company_info']['Company Name']))
    except KeyError:
        data.append(embed(""))
    try:
        data.append(embed(obj['api_data']['job_keywords']))
    except KeyError:
        data.append(embed(""))
    try:
        data.append(embed(obj['api_data']['job_industry']))
    except KeyError:
        data.append(embed(""))
    try:
        data.append(embed(obj['api_data']['job_title']))
    except KeyError:
        data.append(embed(""))
    try:
        data.append(embed(obj['other_details']['Other Skills:']))
    except KeyError:
        data.append(embed(""))
    try:
        data.append(embed(obj['other_details']['Industry:']))
    except KeyError:
        data.append(embed(""))
    try:
        data.append(embed(obj['other_details']['Department:']))
    except KeyError:
        data.append(embed(""))
    try:
        data.append(embed(obj['jd_information']['description']))
    except KeyError:
        data.append(embed(""))
    
    return data
    

In [183]:
##Constructing data for out model

import numpy as np
x_train = []
x_test = []
y_train = []
y_test = []
l_train = []
l_test = []
key_count = 0

for k in description.keys():
    output = np.zeros(len(description.keys()))
    output[key_count] = 1
    label = key_count
    key_count = key_count + 1
    for i in range(len(description[k])):
        with open('data/docs/' + description[k][i] + '.json', 'r') as myfile:
            data=str(myfile.read())
        obj = json.loads(data)
        inp = get_training_data(obj)
#         print(inp)
#         break
#     break
        #70% for training
        if i < 0.7 * len(description[k]):
            
            x_train.append(inp)
            y_train.append(output)
            l_train.append(label)
        #30% for testing
        else:
            x_test.append(inp)
            y_test.append(output)
            l_test.append(label)
            

(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'limitedtfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'limitedtfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'limitedtfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'limitedtfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'limitedtfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'limitedtfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'limitedtfg', 'Not found')
(u

(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'limitedtfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'limitedtfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'limitedtfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'limitedtfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'limitedtfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'tfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'tfg',

(u'bba', 'Not found')
(u'bba', 'Not found')
(u'uniquetravelite', 'Not found')
(u'pressureno', 'Not found')
(u'bossno', 'Not found')
(u'sweta', 'Not found')
(u'tfg', 'Not found')
(u'limitedtfg', 'Not found')
(u'boundation', 'Not found')
(u'tfg', 'Not found')
(u'boundation', 'Not found')
(u'travelite', 'Not found')
(u'travelite', 'Not found')
(u'shahana', 'Not found')
(u'travelite', 'Not found')
(u'travelite', 'Not found')
(u'bba', 'Not found')
(u'bba', 'Not found')
(u'boundation', 'Not found')
(u'bosswe', 'Not found')
(u'infogaurav', 'Not found')
(u'travelite', 'Not found')
(u'travelite', 'Not found')
(u'bba', 'Not found')
(u'bca', 'Not found')
(u'bba', 'Not found')
(u'bca', 'Not found')
(u'boundation', 'Not found')
(u'bosscall', 'Not found')
(u'pawaria', 'Not found')
(u'travelite', 'Not found')
(u'travelite', 'Not found')
(u'bba', 'Not found')
(u'bba', 'Not found')
(u'travelite', 'Not found')
(u'companypart', 'Not found')
(u'usgovind', 'Not found')
(u'travelite', 'Not found')
(u'travel

(u'vishal', 'Not found')
(u'mgrtotal', 'Not found')
(u'lpa', 'Not found')
(u'productmain', 'Not found')
(u'cozmo', 'Not found')
(u'ltdlaunched', 'Not found')
(u'customised', 'Not found')
(u'cozmo', 'Not found')
(u'cozmo', 'Not found')
(u'cozmo', 'Not found')
(u'galileo', 'Not found')
(u'sabre', 'Not found')
(u'amadeus', 'Not found')
(u'galileo', 'Not found')
(u'chargescall', 'Not found')
(u'gunjan', 'Not found')
(u'alisha', 'Not found')
(u'siddharth', 'Not found')
(u'qualityrequirement', 'Not found')
(u'gds', 'Not found')
(u'amadeus', 'Not found')
(u'sabare', 'Not found')
(u'sabre', 'Not found')
(u'amadeus', 'Not found')
(u'galileo', 'Not found')
(u'chargescall', 'Not found')
(u'gunjan', 'Not found')
(u'alisha', 'Not found')
(u'siddharth', 'Not found')
(u'qualityrequirement', 'Not found')
(u'gds', 'Not found')
(u'amadeus', 'Not found')
(u'sabare', 'Not found')
(u'delhiinterested', 'Not found')
(u'ncr', 'Not found')
(u'placenew', 'Not found')
(u'vmake', 'Not found')
(u'\u201c', 'Not fou

(u'mnccompany', 'Not found')
(u'executivesday', 'Not found')
(u'onlylocation', 'Not found')
(u'corpoarte', 'Not found')
(u'immegration', 'Not found')
(u'iata', 'Not found')
(u'galileo', 'Not found')
(u'mumbairemuneration', 'Not found')
(u'benefitsinterested', 'Not found')
(u'angee', 'Not found')
(u'angee', 'Not found')
(u'arseniusskillcapital', 'Not found')
(u'comshare', 'Not found')
(u'benefitarsenius', 'Not found')
(u'sabre', 'Not found')
(u'iata', 'Not found')
(u'amadeus', 'Not found')
(u'galileo', 'Not found')
(u'executivelocation', 'Not found')
(u'andheri', 'Not found')
(u'grossin', 'Not found')
(u'assuredjob', 'Not found')
(u'dealsindia', 'Not found')
(u'afreen', 'Not found')
(u'careermarshal', 'Not found')
(u'comcontact', 'Not found')
(u'afreen', 'Not found')
(u'nishano', 'Not found')
(u'\u201c', 'Not found')
(u'leom', 'Not found')
(u'haryana', 'Not found')
(u'himachal', 'Not found')
(u'uttrakhand', 'Not found')
(u'leom', 'Not found')
(u'zirkpurno', 'Not found')
(u'yearqualifica

...

(u'amritsar', 'Not found')
(u'kabout', 'Not found')
(u'undersignedrgdsanju', 'Not found')
(u'singhbds', 'Not found')
(u'enterprisesbdsconsultants', 'Not found')
(u'anju', 'Not found')
(u'stafflocation', 'Not found')
(u'ahmedabadqualification', 'Not found')
(u'graudatesalary', 'Not found')
(u'experiencekindly', 'Not found')
(u'detailwadalas', 'Not found')
(u'shreeji', 'Not found')
(u'holidaysno', 'Not found')
(u'rulesbonus', 'Not found')
(u'rulespf', 'Not found')
(u'rulesmediclaim', 'Not found')
(u'rulescl', 'Not found')
(u'monthsappraisal', 'Not found')
(u'intervalsdistance', 'Not found')
(u'distanceemployees', 'Not found')
(u'successjobs', 'Not found')
(u'iata', 'Not found')
(u'shreeji', 'Not found')
(u'stafflocation', 'Not found')
(u'ahmedabadqualification', 'Not found')
(u'experiencekindly', 'Not found')
(u'detailwadalas', 'Not found')
(u'shreeji', 'Not found')
(u'holidaysno', 'Not found')
(u'rulesbonus', 'Not found')
(u'rulespf', 'Not found')
(u'rulesmediclaim', 'Not found')
(u'rul

(u'solutionswalk', 'Not found')
(u'gurgaontelephone', 'Not found')
(u'regardsamit', 'Not found')
(u'suthwalhr', 'Not found')
(u'disha', 'Not found')
(u'ltdbackground', 'Not found')
(u'easemytrip', 'Not found')
(u'tryfast', 'Not found')
(u'easemytrip', 'Not found')
(u'easemytrip', 'Not found')
(u'easemytrip', 'Not found')
(u'easemytrip', 'Not found')
(u'easemytrip', 'Not found')
(u'maldives', 'Not found')
(u'mauritius', 'Not found')
(u'easemytrip', 'Not found')
(u'tripadvisor', 'Not found')
(u'easemytrip', 'Not found')
(u'easemytrip', 'Not found')
(u'easemytrip', 'Not found')
(u'nris', 'Not found')
(u'hospitalised', 'Not found')
(u'budapest', 'Not found')
(u'dination', 'Not found')
(u'dination', 'Not found')
(u'hungrybags', 'Not found')
(u'hungrybags', 'Not found')
(u'industryit', 'Not found')
(u'silverline', 'Not found')
(u'clints', 'Not found')
(u'paintmytrip\xae', 'Not found')
(u'paintmytrip', 'Not found')
(u'tourpackages', 'Not found')
(u'aall', 'Not found')
(u'disha', 'Not found')


(u'bookmytournow', 'Not found')
(u'bookmytournow', 'Not found')
(u'bookmytournow', 'Not found')
(u'eelysium', 'Not found')
(u'traveller', 'Not found')
(u'recognised', 'Not found')
(u'bookmytournow', 'Not found')
(u'bookmytournow', 'Not found')
(u'travellers', 'Not found')
(u'bookmytournow', 'Not found')
(u'travellers', 'Not found')
(u'bookmytournow', 'Not found')
(u'\u2019', 'Not found')
(u'bookmytournow', 'Not found')
(u'bookmytournow', 'Not found')
(u'bookmytournow', 'Not found')
(u'\u2019', 'Not found')
(u'bookmytournow', 'Not found')
(u'\u2019', 'Not found')
(u'products\u2022', 'Not found')
(u'hotels\u2022', 'Not found')
(u'bus\u2022', 'Not found')
(u'rentals\u2022', 'Not found')
(u'support\u2022', 'Not found')
(u'insurance\u2022', 'Not found')
(u'bookmytournow', 'Not found')
(u'endtoend', 'Not found')
(u'centre', 'Not found')
(u'environmentexcellent', 'Not found')
(u'centre', 'Not found')
(u'environmentexcellent', 'Not found')
(u'bookmytournow', 'Not found')
(u'bookmytournow', 'No

(u'callsoffer', 'Not found')
(u'tripperceived', 'Not found')
(u'viewholidaytrip', 'Not found')
(u'callsoffer', 'Not found')
(u'tripperceived', 'Not found')
(u'viewholidaytrip', 'Not found')
(u'callsoffer', 'Not found')
(u'tripperceived', 'Not found')
(u'viewholidaytrip', 'Not found')
(u'callsoffer', 'Not found')
(u'marketinglocation', 'Not found')
(u'vikhroli', 'Not found')
(u'mumbaiqualification', 'Not found')
(u'graduateexperience', 'Not found')
(u'yearssalary', 'Not found')
(u'experiencekindly', 'Not found')
(u'detailwadalas', 'Not found')
(u'kruzo', 'Not found')
(u'solutionsno', 'Not found')
(u'rulesbonus', 'Not found')
(u'rulespf', 'Not found')
(u'rulesmediclaim', 'Not found')
(u'rulescl', 'Not found')
(u'monthsappraisal', 'Not found')
(u'intervalsdistance', 'Not found')
(u'distanceemployees', 'Not found')
(u'successjobs', 'Not found')
(u'vikhroli', 'Not found')
(u'kruzo', 'Not found')
(u'marketinglocation', 'Not found')
(u'vikhroli', 'Not found')
(u'mumbaiqualification', 'Not fou

(u'conde', 'Not found')
(u'nast', 'Not found')
(u'conde', 'Not found')
(u'nast', 'Not found')
(u'idseacruiserecruitment', 'Not found')
(u'combest', 'Not found')
(u'regardsrecruitment', 'Not found')
(u'niraj', 'Not found')
(u'ranjan', 'Not found')
(u'idjobcruiserecruiter', 'Not found')
(u'combest', 'Not found')
(u'regardsrecruitment', 'Not found')
(u'niraj', 'Not found')
(u'ranjan', 'Not found')
(u'idseacruiserecruitment', 'Not found')
(u'combest', 'Not found')
(u'regardsrecruitment', 'Not found')
(u'niraj', 'Not found')
(u'ranjan', 'Not found')
(u'kinldy', 'Not found')
(u'upashana', 'Not found')
(u'synergygti', 'Not found')
(u'rupal', 'Not found')
(u'synergygti', 'Not found')
(u'comjob', 'Not found')
(u'onlyapplication', 'Not found')
(u'gof', 'Not found')
(u'mvc', 'Not found')
(u'xdb', 'Not found')
(u'mongodb', 'Not found')
(u'jquery', 'Not found')
(u'couchbase', 'Not found')
(u'memcache', 'Not found')
(u'redissupporting', 'Not found')
(u'jboss', 'Not found')
(u'junit', 'Not found')
(u

In [40]:
len(x_train), len(x_test), len(x_train[0]), len(x_train[0][4]), x_train[0]

(827,
 335,
 9,
 300,
 [array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [46]:
len(y_train),np.array(x_train[0]).shape

(827, (9, 300))

In [184]:
from keras.utils import to_categorical
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
# l_trainn = to_categorical(l_train,31)
# l_testt = to_categorical(l_test,31)
x_train.shape, y_train.shape, type(y_train[0][0]), type(l_trainn),l_trainn.shape

((827, 9, 300), (827, 31), numpy.float64, numpy.ndarray, (827, 31))

### Part II: Exploratoty Data Analysis

_Include EDA steps like finding distribution of Departments in this part, you may also use plots for EDA._

### Part III: Modelling & Evaluation

_Include all model prepration & evaluation steps in this part._

In [177]:
x_train.shape, y_train.shape, type(y_train[0][0])

((827, 9, 300), (827, 31), numpy.float64)

In [178]:
x_train[0].shape, y_train[0]

((9, 300),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [208]:
print(x_train.shape)
x_r = x_train.reshape(827,2700)
xt_r = x_test.reshape(335,2700)
for i in range(len(x_r)):
    if np.isnan(x_r[i].all()):
        print(true)
        break

print("x_train shape ", x_r.shape)
print("x_test shape ", xt_r.shape)
print("y_train shape ", y_train.shape)
print("y_test shape ", y_test.shape)


(827, 9, 300)
('x_train shape ', (827, 2700))
('x_test shape ', (335, 2700))
('y_train shape ', (827, 31))
('y_test shape ', (335, 31))


In [211]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras.regularizers import l2

model = Sequential([
    Dense(1240, input_shape=(2700,),  W_regularizer=l2(0.01)),
    Activation('relu'),
    Dense(124, W_regularizer=l2(0.01)),
    Activation('relu'),
    Dense(31, W_regularizer=l2(0.01)),
    Activation('softmax')
])
opt = optimizers.sgd(lr=0.0001, decay=1e-6)
opt = optimizers.Adam(lr=0.00001, decay=1e-6, clipnorm=1.)

#sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.4, nesterov=False)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

/anaconda3/envs/deeplearningproject/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1240, kernel_regularizer=<keras.reg..., input_shape=(2700,))`
  import sys
/anaconda3/envs/deeplearningproject/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(124, kernel_regularizer=<keras.reg...)`
  if __name__ == '__main__':
/anaconda3/envs/deeplearningproject/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(31, kernel_regularizer=<keras.reg...)`
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
model.fit(x_r, y_train, epochs=1000, batch_size=64,verbose=1)

Epoch 1/1000
827/827 [==============================] - 2s - loss: 23.0868 - acc: 0.1016     
Epoch 2/1000
827/827 [==============================] - 1s - loss: 22.8400 - acc: 0.3821     
Epoch 3/1000
827/827 [==============================] - 1s - loss: 22.5879 - acc: 0.5308     
Epoch 4/1000
827/827 [==============================] - 1s - loss: 22.3336 - acc: 0.6155     
Epoch 5/1000
827/827 [==============================] - 1s - loss: 22.0731 - acc: 0.6348     
Epoch 6/1000
827/827 [==============================] - 1s - loss: 21.8179 - acc: 0.6372     
Epoch 7/1000
827/827 [==============================] - 1s - loss: 21.5676 - acc: 0.6300     
Epoch 8/1000
827/827 [==============================] - 1s - loss: 21.3227 - acc: 0.6252     
Epoch 9/1000
827/827 [==============================] - 1s - loss: 21.0865 - acc: 0.6252     
Epoch 10/1000
827/827 [==============================] - 1s - loss: 20.8550 - acc: 0.6288     
Epoch 11/1000
827/827 [==============================] - 1s

827/827 [==============================] - 1s - loss: 10.3022 - acc: 0.8682     
Epoch 88/1000
827/827 [==============================] - 1s - loss: 10.2135 - acc: 0.8706     
Epoch 89/1000
827/827 [==============================] - 1s - loss: 10.1260 - acc: 0.8706     
Epoch 90/1000
827/827 [==============================] - 1s - loss: 10.0383 - acc: 0.8718     
Epoch 91/1000
827/827 [==============================] - 1s - loss: 9.9505 - acc: 0.8730     
Epoch 92/1000
827/827 [==============================] - 1s - loss: 9.8645 - acc: 0.8730     
Epoch 93/1000
827/827 [==============================] - 1s - loss: 9.7803 - acc: 0.8706     
Epoch 94/1000
827/827 [==============================] - 1s - loss: 9.6957 - acc: 0.8730     
Epoch 95/1000
827/827 [==============================] - 1s - loss: 9.6121 - acc: 0.8730     
Epoch 96/1000
827/827 [==============================] - 1s - loss: 9.5285 - acc: 0.8718     
Epoch 97/1000
827/827 [==============================] - 1s - loss: 9.

827/827 [==============================] - 0s - loss: 5.0879 - acc: 0.9166     
Epoch 174/1000
827/827 [==============================] - 0s - loss: 5.0496 - acc: 0.9166     
Epoch 175/1000
827/827 [==============================] - 0s - loss: 5.0113 - acc: 0.9166     
Epoch 176/1000
827/827 [==============================] - 0s - loss: 4.9737 - acc: 0.9154     
Epoch 177/1000
827/827 [==============================] - 0s - loss: 4.9365 - acc: 0.9154     
Epoch 178/1000
827/827 [==============================] - 0s - loss: 4.8998 - acc: 0.9166     
Epoch 179/1000
827/827 [==============================] - 1s - loss: 4.8639 - acc: 0.9166     
Epoch 180/1000
827/827 [==============================] - 1s - loss: 4.8276 - acc: 0.9166     
Epoch 181/1000
827/827 [==============================] - 1s - loss: 4.7917 - acc: 0.9166     
Epoch 182/1000
827/827 [==============================] - 1s - loss: 4.7564 - acc: 0.9154     
Epoch 183/1000
827/827 [==============================] - 1s - lo

In [192]:
Y_test=model.predict(xt_r)

In [193]:
def precision_recall(gt,preds):
    TP=0
    FP=0
    FN=0
    for t in gt:
        if t in preds:
            TP+=1
        else:
            FN+=1
    for p in preds:
        if p not in gt:
            FP+=1
    if TP+FP==0:
        precision=0
    else:
        precision=TP/float(TP+FP)
    if TP+FN==0:
        recall=0
    else:
        recall=TP/float(TP+FN)
    return precision,recall

In [206]:
score = model.evaluate(xt_r, y_test, batch_size=64)
print("Model metrics: ",model.metrics_names)
print("TEST ACCURACY: ", score[1])

320/335 [===========================>..] - ETA: 0s('Model metrics: ', ['loss', 'acc'])
('TEST ACCURACY: ', 0.9134328363546684)


In [207]:
score

[1.0184799414962085, 0.9134328363546684]

**Results Summary:**
_Provide a brief summary of results obtained like model accuracy & other insights based on EDA & your interpretations_

Key problems that I tackeled were:
1. Data was very unorganized and random, understanding it, preprocessing it and organizing it took a bit of effort.
2. Embeddings played an important role, how to use them to predict the class was something to be thought upon.
3. Extacted important features from the data and removed the redundant features on my own.
4. Major problem was the missing data and some words not being present in the vocabulary of word model. Missing data started to give me nan loss from the first iteration. For missing data, I aim to go throught this paper: http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.23.6971 . As of now, I returned a very small sentence vector if data is missing.
5. I was planing to use universal sentence encoder but that was too big for me to download at my home, as of now, I just used word2vec embeddings and averaged out the embeddings for a sentence.
6. Instead of using random small vector, incorporate an unknown vector.

1. Got testing accuracy to be around 91.3 % which is on 335 samples and I used 827 samples in total to train the model.
2. Loss is around 1.01 on test data. USed softmax in last layer.
3. 